In [105]:
import psycopg2
import pandas as pd
from jupyterthemes import jtplot
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
conn = psycopg2.connect("dbname=postgres user=postgres password=your-password-here host=172.17.0.2")
cur = conn.cursor()

# get maximum extent for each market

## by provider?

In [16]:
def rollback ():
    conn.commit()

In [54]:
def get_market (market_name):
    '''
    Gets all data on market_name YTD
    '''
    q = f'''
    SELECT
      name,
      jurisdiction_alpha2,
      marketshare
    FROM provider_marketshare
    WHERE
      time between '2021-01-01' and '2021-06-20' AND
      market = '{market_name}' AND
      marketshare > 0 AND
      jurisdiction_alpha2 is not NULL
    '''
    rollback()
    cur.execute(q)
    results = cur.fetchall()
    market = pd.DataFrame(results, columns=['name', 'jurisdiction_alpha2', 'marketshare'])
    market.marketshare = market.marketshare.astype(float)
    return market

In [55]:
m = get_market('dns-server')

In [57]:
m.groupby('name').mean()

,marketshare
name,
1984 Hosting,0.000110
A2 Hosting,0.003256
AT&T,0.000375
Aerotek,0.001997
Afraid,0.000315
...,...
Yandex,0.004747
cdmon,0.000996
easyDNS Group,0.001006


In [94]:
markets = [
    'dns-server',
    'proxy',
    'ssl-certificate',
    'data-centers',
#     'server-location',
    'web-hosting',
    'top-level-domain',
]

ms = []
for market in markets:
    m = get_market(market).groupby('name').mean() 
    m = m.rename({'marketshare': market}, axis=1)
    ms.append(m)
    

In [95]:
t = ms[0]
for i, m_df in enumerate(ms[1:]):
    t = t.merge(ms[i+1], how='outer', left_on='name', right_on='name')

In [97]:
t

,dns-server,proxy,ssl-certificate,data-centers,web-hosting,top-level-domain
name,,,,,,
1984 Hosting,0.000110,NaN,NaN,0.000035,0.000035,NaN
A2 Hosting,0.003256,NaN,NaN,0.004003,0.004003,NaN
AT&T,0.000375,NaN,NaN,0.000985,0.000989,NaN
Aerotek,0.001997,NaN,NaN,0.002996,0.002996,NaN
Afraid,0.000315,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
.uz,NaN,NaN,NaN,NaN,NaN,0.001000
.vn,NaN,NaN,NaN,NaN,NaN,0.009119
.xyz,NaN,NaN,NaN,NaN,NaN,0.003000


In [99]:
max_extent = t.max(axis=1).sort_values(ascending=False)

In [101]:
max_extent[:20].plot.bar()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

## for each provider?

In [21]:
q = '''
SELECT
  name AS metric,
  sum(marketshare) AS "marketshare"
FROM provider_marketshare
WHERE
  time between '2020-01-01' and '2021-06-20' AND
  market = 'ssl-certificate' AND
  marketshare > 0 AND
  name is not NULL
GROUP BY 1
ORDER BY 1,2
'''
rollback()
cur.execute(q)
cur.fetchall()

[('Actalis', Decimal('0.1988399999999999998')),
 ('AffirmTrust', Decimal('0.00053')),
 ('Buypass', Decimal('0.00139000000000000006')),
 ('Camerfirma', Decimal('0.00112')),
 ('Certigna', Decimal('0.00065')),
 ('CertSIGN', Decimal('0.00014')),
 ('Certum', Decimal('0.9598400000000000035')),
 ('Chunghwa Telecom', Decimal('0.00206000000000000002')),
 ('Deutsche Telekom', Decimal('0.00450000000000000005')),
 ('DigiCert Group', Decimal('37.20833999999999930')),
 ('D-Trust', Decimal('0.00035')),
 ('Entrust', Decimal('0.3808899999999999999')),
 ('E-Tugra', Decimal('0.1851000000000000001')),
 ('GlobalSign', Decimal('5.233610000000000280')),
 ('GoDaddy Group', Decimal('12.38021000000000000')),
 ('Harica', Decimal('0.00001')),
 ('Hongkong Post', Decimal('0.00101')),
 ('IdenTrust', Decimal('100.3468599999999982')),
 ('Izenpe', Decimal('0.00027')),
 ('Let’s Encrypt', Decimal('0.6832500000000000056')),
 ('Logius', Decimal('0.00124')),
 ('Microsec', Decimal('0.00015')),
 ('NetLock', Decimal('0.00099')